In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# - YouTube Data API를 사용해
# - 단일 영상의 메타데이터를 수집한다
# - (title, view_count, like_count, channel info)

def create_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")

    options.add_argument("--start-maximized")
    options.add_argument("disable-dpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=ko-KR")

    service =  Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver
driver = create_driver(headless=False)

In [8]:
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # 테스트용
driver.get(video_url)



In [18]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="00c7293f16fdbe5c13e8278ecf31bc3b")>

In [19]:
driver.title


'Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster) - YouTube'

In [ ]:
# 필요 라이브러리 선언
import requests
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs
import os
import time
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [ ]:
# 수집 할 영상 링크 리스트
video_urls = [
    # 이세계 아이돌 
    "https://www.youtube.com/watch?v=k5O1F_IS0mU",
    "https://www.youtube.com/watch?v=rDFUl2mHIW4",
    "https://www.youtube.com/watch?v=fff8P0kYexQ",

    # 플레이브
    "https://www.youtube.com/watch?v=T_gJpnjJsnk",
    "https://www.youtube.com/watch?si=x1kGDm5IWkb8dLXU&v=EYG4ROejmyI&feature=youtu.be",
    "youtube.com/watch?si=y1keUKV5tnDV5V99&v=8L001YT4k6k&feature=youtu.be",

    # 아이브
    "https://www.youtube.com/watch?v=bltjk6NfDQM",
    "https://www.youtube.com/watch?si=1nYu7Hlvgl8eRGgQ&v=6ZUIwj3FgUY&feature=youtu.be",
    "https://www.youtube.com/watch?si=pA-RrdGj--kfVG_A&v=B1ShLiq3EVc&feature=youtu.be",

    # RIIZE
    "youtube.com/watch?si=qeXcW7SgUNtbaIP1&v=3_b5oKGndn0&feature=youtu.be",
    "https://www.youtube.com/watch?si=jnfaeWEXYL_ZWpyW&v=iUw3LPM7OBU&feature=youtu.be",
    "youtube.com/watch?si=cCUEC_2ivRFY7CrV&v=TVqEwvbZP30&feature=youtu.be"
]


In [ ]:
# 영상 링크에서 영상 ID 가져오는 함수

# Youtube API key 값 .env에서 가져오기
load_dotenv()

def extract_video_id(url: str) -> str | None:
    parsed = urlparse(url)

    # youtu.be 형태
    if parsed.netloc == "youtu.be":
        return parsed.path.lstrip("/")

    # youtube.com/watch 형태
    query = parse_qs(parsed.query)
    return query.get("v", [None])[0]    

In [ ]:
# 기본 url
url = "https://www.googleapis.com/youtube/v3/videos"

# video url 리스트에서 video id 값 가져오기
video_ids = [
    extract_video_id(u) for u in video_urls
    if extract_video_id(u) is not None
]

# video id 중복된 영상 ID 제거
unique_video_ids = list(set(video_ids))

data_list = []
# unique_video_ids에 포함된 각 video_id에 대해 영상 메타데이터를 API로 요청
for i in unique_video_ids:
    # YouTube Data API 요청에 사용할 파라미터 설정
    params = {
        "part": "snippet,statistics",
        "id": i,
        "key": os.getenv("YOUTUBE_API_KEY")
    }
    # video list의 각 video API 요청
    response = requests.get(url, params=params)

    # API 응답을 JSON 형태로 변환
    data = response.json()
    print(response)

    # 각 영상에 대한 API 응답(JSON)을 data_list에 저장
    data_list.append(data)

print(data_list)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
[{'kind': 'youtube#videoListResponse', 'etag': 'g96qa8mV4odlh-WTrl155n4whFk', 'items': [{'kind': 'youtube#video', 'etag': 'a2UFjqp4mnmXKiirV9rouB27Iso', 'id': 'bltjk6NfDQM', 'snippet': {'publishedAt': '2021-12-09T08:00:22Z', 'channelId': 'UCS_hnpJLQTvBkqALgapi_4g', 'title': "[단독샷캠] 아이브 'ELEVEN' 단독샷 별도녹화│IVE ONE TAKE STAGE│@SBS Inkigayo_2021.12.05.", 'description': "[단독샷캠] 아이브 'ELEVEN' 단독샷 별도녹화│IVE ONE TAKE STAGE│@SBS Inkigayo_2021.12.05.\n(Ft. 스탠다드 별도녹화)\n\n#인기가요 #아이브 #ELEVEN\n---------------------------------------------------------------------------------------—\n\n☞ MORE ＂SBS KPOP＂\n\n- Twitter : https://twitter.com/kpop_sbs\n- Facebook : https://www.facebook.com/sbskpop\n- Instagram : https://instagram.com/sbskpop_official", 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi

In [ ]:
# 수집된 video_ids 데이터 확인
Counter(video_ids)

In [112]:
len(data_list[0]["items"])

1

In [ ]:

video_list = [] 

# data_list에 저장된 각 영상의 API 응답(JSON)을 순회
for data in data_list:

    # API 응답에서 영상 정보가 담긴 items 리스트 추출
    items = data.get("items", [])
    items

    # items가 비어 있으면 (잘못된 video_id, 비공개/삭제 영상 등)
    if not items:
        print("No item found")
        item = None
    else:
        # videos API는 id 하나만 요청했으므로 첫 번째 요소 사용
        item = items[0]

    snippet = item.get("snippet", {})
    statistics = item.get("statistics", {})
    raw_view = statistics.get("viewCount")
    raw_like = statistics.get("likeCount")

    view_count = int(raw_view) if raw_view is not None else None
    like_count = int(raw_like) if raw_like is not None else None

    video_meta = {
        "video_id": item.get("id"),
        "channel_id": snippet.get("channelId"),
        "published_at": snippet.get("publishedAt"),
        "video_title": snippet.get("title"),
        "channel_name": snippet.get("channelTitle"),
        "view_count": view_count,
        "like_count": like_count,
        "commentCount":statistics.get("commentCount")
    }
    
    video_list.append(video_meta)



In [ ]:
# 특정 영상의 댓글을 YouTube Data API로 수집하는 함수
def fetch_comments(video_id, max_comments=200):
    comments = []                     # 수집된 댓글을 저장할 리스트
    page_token = None                 # pagination을 위한 토큰 (초기값 None)
    
    while True:
        params = {
            "part": "snippet",
            "videoId": video_id,
            "maxResults": 100,        # API에서 허용하는 최대 댓글 수
            "order": "time",          # 최신 댓글순으로 조회
            "textFormat": "plainText",
            "key": os.getenv("YOUTUBE_API_KEY")
        }
        # 다음 페이지가 존재할 경우 pageToken 추가
        if page_token:
            params["pageToken"] = page_token

        # 댓글 조회 API 요청
        resp = requests.get(
            "https://www.googleapis.com/youtube/v3/commentThreads",
            params=params
        )

        # 디버깅 및 상태 확인용 로그
        print("STATUS CODE:", resp.status_code)
        print("RAW RESPONSE:", resp.text)

        data = resp.json()
      
        # 댓글 비활성화, 삭제 영상, 권한 문제 등 오류 처리
        if "error" in data:
            print(f"[WARN] comments disabled or error for video_id={video_id}")
            break
        
        # 각 댓글(thread)의 top-level comment 추출
        for item in data.get("items", []):
            snippet = item.get("snippet", {})
            top = snippet.get("topLevelComment", {})
            c_snippet = top.get("snippet", {})

            # 댓글 메타데이터 정리
            comments.append({
                "video_id": video_id,
                "comment_id": top.get("id"),
                "comment_text": c_snippet.get("textDisplay"),
                "like_count": c_snippet.get("likeCount"),
                "published_at": c_snippet.get("publishedAt")
            })
            # 설정한 최대 댓글 수에 도달하면 즉시 반환
            if len(comments) >= max_comments:
                return comments
            
        # 다음 페이지 토큰 추출
        page_token = data.get("nextPageToken")

        # 다음 페이지가 없으면 반복 종료
        if not page_token:
            break
        
        # API quota 및 안정성을 위한 대기
        time.sleep(0.1) 
        
    # 수집된 댓글 반환
    return comments

In [ ]:
# 모든 영상의 댓글을 하나로 모아 저장할 리스트
all_comments = []

# video_list에 포함된 각 영상에 대해 댓글 수집
for video in video_list:
    # 현재 영상의 video_id 추출
    vid = video["video_id"]     

    # 해당 영상의 댓글을 최대 150개까지 수집
    comments = fetch_comments(vid, max_comments=150)
    
    # 수집된 댓글들을 전체 댓글 리스트에 누적
    all_comments.extend(comments)


STATUS CODE: 200
RAW RESPONSE: {
  "kind": "youtube#commentThreadListResponse",
  "etag": "810ZETFgYGl6s9HjsKWaUhkzzJY",
  "nextPageToken": "Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNJZ2dHQUFTQlFpSElCZ0FFZ1VJblNBWUFSSUZDSWtnR0FBU0JRaW9JQmdBSWc0S0RBajN0WXpCQmhDb3k1dXJBUQ==",
  "pageInfo": {
    "totalResults": 100,
    "resultsPerPage": 100
  },
  "items": [
    {
      "kind": "youtube#commentThread",
      "etag": "C0zZsuXH8W1rfnrLle8pJH-70Yo",
      "id": "Ugykjh0_hVuTgqN8doR4AaABAg",
      "snippet": {
        "channelId": "UCzh4yY8rl38knH33XpNqXbQ",
        "videoId": "k5O1F_IS0mU",
        "topLevelComment": {
          "kind": "youtube#comment",
          "etag": "mvXnywk8-0tjNdGLtPlhjkRwXPk",
          "id": "Ugykjh0_hVuTgqN8doR4AaABAg",
          "snippet": {
            "channelId": "UCzh4yY8rl38knH33XpNqXbQ",
            "videoId": "k5O1F_IS0mU",
            "textDisplay": "그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트 가본 아이돌도 너네가 처음이라서 진짜 익숙하지 않지만 내가 좋아하는 마음은 내 자신도

(150,
 [{'video_id': 'k5O1F_IS0mU',
   'comment_id': 'Ugykjh0_hVuTgqN8doR4AaABAg',
   'comment_text': '그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고 콘서트 가본 아이돌도 너네가 처음이라서 진짜 익숙하지 않지만 내가 좋아하는 마음은 내 자신도 상상하는 만큼보다 크다는 거 ㅠㅠㅠㅠ 진짜 너무 너무 사랑해 애정해',
   'like_count': 0,
   'published_at': '2026-01-31T15:59:29Z'},
  {'video_id': 'k5O1F_IS0mU',
   'comment_id': 'Ugx1c4OKEnkVwGWBlFB4AaABAg',
   'comment_text': '눈물이 난다',
   'like_count': 2,
   'published_at': '2026-01-23T04:29:25Z'},
  {'video_id': 'k5O1F_IS0mU',
   'comment_id': 'UgwaAbmKOm2vwOgFEFp4AaABAg',
   'comment_text': '이 노래도 겁나 좋네',
   'like_count': 2,
   'published_at': '2026-01-20T01:52:56Z'}])

In [ ]:
# 특정 영상의 모든 댓글을 YouTube Data API로 가능한 범위까지 수집하는 함수
def fetch_all_comments(video_id, sleep_sec=0.2):
    comments = []           # 수집된 모든 댓글을 저장할 리스트
    page_token = None       # pagination을 위한 페이지 토큰
    page_count = 0          # 수집된 페이지 수 (로그/디버깅용)

    # 다음 페이지가 없을 때까지 반복
    while True:
        # commentThreads API 요청 파라미터 설정
        params = {
            "part": "snippet",
            "videoId": video_id,
            "maxResults": 100,              # API에서 허용하는 최대 댓글 수
            "order": "time",                # 최신 댓글 순으로 조회
            "textFormat": "plainText",
            "key": os.getenv("YOUTUBE_API_KEY")
        }

        # 다음 페이지가 존재할 경우 pageToken 추가
        if page_token:
            params["pageToken"] = page_token

        # 댓글 조회 API 요청
        resp = requests.get(
            "https://www.googleapis.com/youtube/v3/commentThreads",
            params=params
        )

        # API 응답을 JSON 형태로 변환
        data = resp.json()

        # 댓글 비활성화, 삭제 영상, 권한 문제 등 오류 처리
        if "error" in data:
            print(f"[WARN] skip video_id={video_id}")
            break

        # 현재 페이지의 댓글 목록 추출    
        items = data.get("items", [])

        # 더 이상 댓글이 없으면 반복 종료
        if not items:
            break
        
        # 각 댓글(thread)의 top-level comment 추출
        for item in items:
            top = item["snippet"]["topLevelComment"]
            snippet = top["snippet"]

            # 댓글 메타데이터 저장
            comments.append({
                "video_id": video_id,
                "comment_id": top["id"],
                "comment_text": snippet["textDisplay"],
                "like_count": snippet["likeCount"],
                "published_at": snippet["publishedAt"]
            })

        # 수집 진행 상황 로그 출력
        page_count += 1
        print(f"page {page_count} | total comments: {len(comments)}")

        # 다음 페이지 토큰 추출
        page_token = data.get("nextPageToken")

        # 다음 페이지가 없으면 반복 종료
        if not page_token:
            break
        
        # API quota 및 요청 안정성을 위한 대기        
        time.sleep(sleep_sec)
        
    # 수집된 모든 댓글 반환
    return comments


In [ ]:
# 모든 영상의 댓글을 하나의 리스트로 수집
all_comments = []

# video_list에 포함된 각 영상을 순회하며 댓글 수집
for i, video in enumerate(video_list, 1):
    video_id = video["video_id"]            # 현재 영상의 video_id
    title = video.get("video_title")        # 영상 제목 (로그 출력용)

    # 진행 상황 및 현재 수집 중인 영상 정보 출력
    print(f"\n[{i}/{len(video_list)}] Fetching comments for: {title}")

    # 해당 영상의 모든 댓글을 API가 허용하는 범위까지 수집
    comments = fetch_all_comments(video_id)

    # 수집된 댓글 개수 출력
    print(f"→ collected {len(comments)} comments")
    
    # 수집된 댓글을 전체 댓글 리스트에 누적
    all_comments.extend(comments)



[1/12] Fetching comments for: [단독샷캠] 아이브 'ELEVEN' 단독샷 별도녹화│IVE ONE TAKE STAGE│@SBS Inkigayo_2021.12.05.
page 1 | total comments: 100
page 2 | total comments: 200
page 3 | total comments: 300
page 4 | total comments: 400


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df_comments = pd.DataFrame(all_comments)
df_comments.head()
df_video = pd.DataFrame(video_list)
df_video.head()

,video_id,channel_id,published_at,video_title,channel_name,view_count,like_count,commentCount
0,bltjk6NfDQM,UCS_hnpJLQTvBkqALgapi_4g,2021-12-09T08:00:22Z,[단독샷캠] 아이브 'ELEVEN' 단독샷 별도녹화│IVE ONE TAKE STAG...,SBSKPOP X INKIGAYO,13292233,162030,1827
1,8L001YT4k6k,UCPZIPuQPrfrUG9Xe_okEmQA,2025-11-10T09:00:07Z,PLAVE 'BBUU!' M/V,PLAVE 플레이브,14145241,102910,9739
2,iUw3LPM7OBU,UCEf_Bc-KVd7onSeifS3py9g,2023-09-04T09:00:24Z,RIIZE 라이즈 'Get A Guitar' MV,SMTOWN,57056199,689477,42047
3,B1ShLiq3EVc,UCYDmx2Sfpnaxg488yBpZIGg,2025-08-25T09:00:05Z,IVE 아이브 ‘XOXZ’ MV,STARSHIP,34756422,536696,31591
4,k5O1F_IS0mU,UCzh4yY8rl38knH33XpNqXbQ,2023-12-21T17:36:37Z,이세계 페스티벌 - RE:WIND,왁타버스 WAKTAVERSE,1146206,23481,2225


In [ ]:
# 영상 리스트내 모든 영상의 메타데이터 저장
df_video.to_csv(
    "../data/youtube_video_meta.csv",
    index=False,
    encoding="utf-8-sig"
)

In [ ]:
# 각 영상의 모든 댓글 메타데이터 저장
df_comments.to_csv(
    "../data/youtube_comments.csv",
    index=False,
    encoding="utf-8-sig"
)

In [5]:
# 같은 폴더에 있는 CSV 파일을 읽는 방식
df = pd.read_csv("../data/youtube_comments.csv")

df.head()


,video_id,comment_id,comment_text,like_count,published_at
0,k5O1F_IS0mU,Ugykjh0_hVuTgqN8doR4AaABAg,그냥 얘들아 내가 진짜 ㅠ 애정한다 ㅠ 나 이렇게 애정해본 아이돌이 너네가 처음이고...,0,2026-01-31T15:59:29Z
1,k5O1F_IS0mU,Ugx1c4OKEnkVwGWBlFB4AaABAg,눈물이 난다,2,2026-01-23T04:29:25Z
2,k5O1F_IS0mU,UgwaAbmKOm2vwOgFEFp4AaABAg,이 노래도 겁나 좋네,2,2026-01-20T01:52:56Z
3,k5O1F_IS0mU,UgzquReVBuSocVLtI7t4AaABAg,"패러블, 이제 뉴세돌로 리와인드 다시 찍어 올려라!!! 이세돌 더 이쁜 버전으로 보...",3,2026-01-19T11:10:22Z
4,k5O1F_IS0mU,UgwM02x2Puib48rQTuF4AaABAg,언제 들어도 행복하고 좋은 노래..,2,2026-01-16T02:53:32Z


In [6]:
df["video_id"].unique()

<StringArray>
['k5O1F_IS0mU', 'rDFUl2mHIW4', 'fff8P0kYexQ', 'T_gJpnjJsnk', 'EYG4ROejmyI',
 '8L001YT4k6k', '6ZUIwj3FgUY', 'B1ShLiq3EVc', '3_b5oKGndn0', 'iUw3LPM7OBU',
 'TVqEwvbZP30']
Length: 11, dtype: str

In [74]:
# 이미 저장된 댓글 CSV 로드
df_comments = pd.read_csv("../data/youtube_comments.csv")

# 메타에 있는 전체 video_id
meta_video_ids = set(v["video_id"] for v in video_list)

# 댓글 CSV에 실제 존재하는 video_id
comment_video_ids = set(df_comments["video_id"].unique())

# 빠진 video_id
missing_video_ids = meta_video_ids - comment_video_ids
missing_video_ids

set()

In [86]:
TARGET_URL = "youtube.com/watch?si=RZf9EA9TSPukVqbf&v=bltjk6NfDQM&feature=youtu.be"
TARGET_VIDEO_ID = extract_video_id(TARGET_URL)

TARGET_VIDEO_ID

'bltjk6NfDQM'

In [87]:


df_comments = pd.read_csv("../data/youtube_comments.csv")

TARGET_VIDEO_ID in set(df_comments["video_id"])


False

In [88]:

missing_comments = fetch_all_comments(TARGET_VIDEO_ID)

len(missing_comments)

page 1 | total comments: 100
page 2 | total comments: 200
page 3 | total comments: 300
page 4 | total comments: 400
page 5 | total comments: 500
page 6 | total comments: 600
page 7 | total comments: 700
page 8 | total comments: 800
page 9 | total comments: 900
page 10 | total comments: 1000
page 11 | total comments: 1100
page 12 | total comments: 1200
page 13 | total comments: 1300
page 14 | total comments: 1400
page 15 | total comments: 1500
page 16 | total comments: 1512


1512

In [101]:
print(missing_comments)

[{'video_id': 'bltjk6NfDQM', 'comment_id': 'Ugw-HcESiZIXJI_2n0d4AaABAg', 'comment_text': 'Di sini Leeseo masih kecil bangeeettt 😭 cimit gemashhh', 'like_count': 0, 'published_at': '2026-02-03T01:18:15Z'}, {'video_id': 'bltjk6NfDQM', 'comment_id': 'Ugw1SlNZWN9Z8DxRGeV4AaABAg', 'comment_text': '저거다 ~ 기계라서 기계 한테 보나스 줘야함', 'like_count': 0, 'published_at': '2026-02-01T02:41:28Z'}, {'video_id': 'bltjk6NfDQM', 'comment_id': 'UgxkGWBOmpTcknO0dnd4AaABAg', 'comment_text': '伝説の始まり', 'like_count': 0, 'published_at': '2026-01-28T17:10:22Z'}, {'video_id': 'bltjk6NfDQM', 'comment_id': 'UgzrPBnqF7GgEkofxzV4AaABAg', 'comment_text': '혹시 2026에 있는 사람', 'like_count': 9, 'published_at': '2026-01-27T02:34:30Z'}, {'video_id': 'bltjk6NfDQM', 'comment_id': 'Ugw_uSrn9L9LSPizdyx4AaABAg', 'comment_text': 'THESE WOMEEEEEEEEEEEEEEEN 🥵💕💕💕💕💕💕💕💕💕💕💕💕💕💕💕💕💕💕', 'like_count': 0, 'published_at': '2026-01-23T18:17:27Z'}, {'video_id': 'bltjk6NfDQM', 'comment_id': 'UgzpHTDKRj72qCnd7Tt4AaABAg', 'comment_text': '이때는 이서가 레전드였네', '

In [105]:
# 기존 CSV
df_comments = pd.read_csv("../data/youtube_comments.csv")

# 새로 수집한 댓글
df_missing = pd.DataFrame(missing_comments)

# 병합
df_merged = pd.concat(
    [df_comments, df_missing],
    ignore_index=True
)

# CSV 저장 (덮어쓰기)
df_merged.to_csv(
    "../data/youtube_comments.csv",
    index=False,
    encoding="utf-8-sig"
)